# 🧮 Calculations for spheres moving through fluids
This worksheet provides some computational tools to help you understand and use the ideas of scale models and dynamic similarity, as implemented using the Reynolds number, $\mathcal{Re}$, and the Coefficient of Drag, $C_d$. The rationale for why these nondimensional indices are useful and concise tools for quantifying 

In [1]:
# Import modules:
#%matplotlib
from math import *
import numpy as np
from scipy.interpolate import CubicSpline, interp1d
from scipy.optimize import root, bisect
import matplotlib.pyplot as plt
plt.ion()
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
# Experimental data for Cd for a sphere:
relist=[ .05875, .1585, .4786, 3.020, 7.015, 15.49, 57.54, 144.5, 264.9, 512.9, 1000., 1862., 3162., 4764., 8375., .1556*10**5, .2648*10**5, .3467*10**5, .5888*10**5, .1000*10**6, .1702*10**6, .2317*10**6, .2648*10**6, .2710*10**6, .2851*10**6, .3020*10**6, .3388*10**6, .3981*10**6, .5129*10**6, .1778*10**7, .2291*10**7, .5012*10**7]
cdlist = [492.0, 169.8, 58.88, 10.86, 5.623, 3.388, 1.479, .9204, .7194, .5623, .4786, .4365, .4074, .3890, .3981, .4395, .4571, .4775, .4732, .4624,.4395, .4046, .3733, .3467, .2472, .1778, .1047, .09772, .1000, .1778, .1862, .1862]
CdRe_logdata = [(log(relist[i],10),log(cdlist[i],10)) for i in range(len(relist))]
Re_logdata = [log(relist[i],10) for i in range(len(relist))]
Cd_logdata = [log(cdlist[i],10) for i in range(len(relist))]
stokes_transition = 1.5
#CdL_interp = spline(CdRe_logdata)
#CdL_interp = CubicSpline(Re_logdata,Cd_logdata)
CdL_interp = interp1d(Re_logdata,Cd_logdata)
ReL_plot = np.linspace(-4,Re_logdata[-1],num=128)
#ReL_plot = np.linspace(Re_logdata[0],Re_logdata[-1],num=128)
#CdL_plot = CdL_interp(ReL_plot)
# Stokes and Oseen analytical solutions
Cd_Stokes = lambda Re_: 24./Re_
Cd_Oseen = lambda Re_: 24./Re_*(1.+3./16.*Re_)
CdL_Stokes = lambda logRe_: np.log10(24./np.power(10,logRe_))
CdL_Oseen = lambda logRe_: np.log10(24./np.power(10,logRe_)*(1.+3./16.*np.power(10,logRe_)))
CdRe = lambda logRe_: CdL_Oseen(logRe_) if logRe_ <= log(stokes_transition,10) else CdL_interp(logRe_)
CdL_plot = [CdRe(r) for r in ReL_plot]

## Matching Reynolds numbers for dynamic similarity
This part of the worksheet enables you to calculate the parameters to make a dynamically similar scale model of an organism moving in a fluid.
The input panel below has two columns. 
In the left column, you can enter the characteristics of the object of interest, and the fluid in which it is immersed. In the right column, you can enter the characteristics of the model organism and the fluid in which it is immersed. 
The worksheet uses the formula,
$$
\mathcal{Re} = \frac{\rho U L}{\mu},
$$
to calculate the Reynolds numbers of both objects for you. 
If they match, the model is dynamically similar to the object.

In [3]:
LabelObj = widgets.Label(value='Object',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="center", 
                                               width="100%"))
LabelModel = widgets.Label(value='Model',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="center", 
                                               width="100%"))
mu1=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho1=widgets.FloatText(value=1028.,description = "$\\rho$ ($kg/m^3$)")
U1=widgets.FloatText(value=5.e-6,description = "$U$ ($m/s$)")
L1=widgets.FloatText(value=5.e-6,description = "$L$ (m)")

mu2=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho2=widgets.FloatText(value=1028.,description = "$\\rho$ ($kg/m^3$)")
U2=widgets.FloatText(value=5.e-6,description = "$U$ ($m/s$)")
L2=widgets.FloatText(value=5.e-6,description = "$L$ (m)")

ui1 = widgets.VBox([LabelObj,mu1,rho1,U1,L1])
ui2 = widgets.VBox([LabelModel,mu2,rho2,U2,L2])
ui3 = widgets.HBox([ui1, ui2])

def REfunc1(rho1,U1,L1,mu1):
    print('Object: Re = {:.4g}'.format(abs(rho1*U1*L1/mu1)))

def REfunc2(rho2,U2,L2,mu2):
    print('Model: Re = {:.4g}'.format(abs(rho2*U2*L2/mu2)))

out1 = widgets.interactive_output(REfunc1,{'rho1':rho1,'U1':U1,'L1':L1,'mu1':mu1})
out2 = widgets.interactive_output(REfunc2,{'rho2':rho2,'U2':U2,'L2':L2,'mu2':mu2})
display(ui3,out1,out2)

Output()

Output()

:::{figure} #rs1_1
:placeholder: ./Images/RS1_1.png
:::

## Calculating force on a spherical particle moving at known velocity, $U$
### Force calculator
This part of the worksheet enables you to calculate the force required to propel an organism of given size through a fluid with known viscosity and density. 
The rationale is as follows:
1. Use the organism's diameter, $D$ and velocity, $U$, and the fluid density ($\rho$) and viscosity ($\mu$), to calculate $\mathcal{Re}$.
2. Use $\mathcal{Re}$ to calculate the Coefficient of Drag, $C_d$,
   $$
    C_d  = \frac{F}{\frac{1}{2} \rho L^2 U^2}
   $$
3. Use $C_d$, the organism's size and velocity, and the fluid density, to calculate the drag force $F$. To do this, we need to [algebraically rearrange the formula for $C_d$ to solve for $F$](./CdSphere.ipynb).

The input panel below has text boxes for the particle size and velocity and the fluid density and viscosity as inputs, and the force required to maintain the particle's velocity as output:

In [4]:
mu3=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho3=widgets.FloatText(value=1028.,description = r"$\rho~~ (\frac{kg}{m^3})$")
U3=widgets.FloatText(value=5.e-6,description = r"$U~~ (\frac{m}{s})$")
D3=widgets.FloatText(value=5.e-6,description = r"$D~~ (m)$")

ui1a = widgets.VBox([mu3,rho3])
ui2a = widgets.VBox([U3,D3])
ui3a = widgets.HBox([ui1a, ui2a])

def F3(rho3,U3,D3,mu3):
    Re=abs(D3*U3*rho3/mu3)
    logRe=log(Re,10)
    logCd=CdRe(logRe)
    Cd_=10**logCd
    F_=np.sign(U3)*0.5*rho3*pi/4.*D3**2*U3**2*Cd_
    print('Force required to move sphere at velocity U: F = {:.4g} N'.format(F_))
     
out3 = widgets.interactive_output(F3,{'rho3':rho3,'U3':U3,'D3':D3,'mu3':mu3})
display(ui3a,out3)

Output()

:::{figure} #rs1_1
:placeholder: Images/RS1_2.png
:::

### Density calculator
This calculator addresses the situation when we have observed a particle's sinking rate, and we would like to infer its density.
This is closely related to the situation above:

Having calculated the force $F$, we then need to do an additional calculation to find the density of the sphere required to produce that force on the immersed particle. 
In this calculator, the inputs are the same as above, and the output is the density required to move the particle at the specified velocity.

In [5]:
g=9.81

mu4=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho_water=widgets.FloatText(value=1028.,description = r"$\rho_{water}~~ (\frac{kg}{m^3})$")
U4=widgets.FloatText(value=5.e-6,description = r"$U~~ (\frac{m}{s})$")
D4=widgets.FloatText(value=5.e-6,description = r"$D~~ (m)$")

ui1b = widgets.VBox([mu4,rho_water])
ui2b = widgets.VBox([U4,D4])
ui3b = widgets.HBox([ui1b, ui2b])

def rho4(rho_water,U4,D4,mu4):
    if U4==0: # We don't have to calculate anything...
        rho_sphere=rho4
        print('Particle density equals fluid density: rho_sphere = {:.4g}'.format(rho_sphere))
    else:
        # Geometry:
        rr=D4/2.
        vol=float(4./3.*pi*rr**3)
        Re=abs(D4*U4*rho_water/mu4)
        logRe=log(Re,10)
        logCd=CdRe(logRe)
        Cd_=10**logCd
        F_=np.sign(U4)*0.5*rho_water*pi/4.*D4**2*U4**2*Cd_
        # Density calculation
        excess_mass=F_/g
        rho_excess=excess_mass/vol
        rho_sphere=rho_excess+rho_water     
        print('Particle density: rho_sphere =  = {:.4g} kg/m^3'.format(rho_sphere))
    
out4 = widgets.interactive_output(rho4,{'rho_water':rho_water,'U4':U4,'D4':D4,'mu4':mu4})
display(ui3b,out4)

Output()

:::{figure} #rs1_1
:placeholder: Images/RS1_3.png
:::

## Calculating velocity of a spherical particle propelled by known force, $F$
### Velocity calculator #1
As noted, the $\mathcal{Re}-C_d$ curve describes the movement of all spheres at all speeds in all [Newtonian fluids](wiki:Newtonian_fluid) like air and water. 
This is sufficient to determine the velocity of a sphere if we know its size, the force exerted on it, and the fluid characteristics. 

There is however a slight complication: 
Both $Re$ and $C_d$ are functions of particle velocity, $U$, (see the equations above). 
However, because we don't in general have a convenient formula for $C_d$, we can't write down an analytical formula to obtain the $U$ that satisfies both these equations simulataneously.

To get past this hurdle, we'll use a time-honored mathematical technique: We'll guess.

Guessing, or more precisely developing an intelligent sequence of trial-and-error iterations, is a great way to solve many computational problems. 
In fact, this type of iteration is essentially what a lot of computer algorithms for solving many hard problems are doing. 
Below, I've made a calculator that does this iteration for you. 

Before trusting in the software, however, it's important that you gain some first-hand experience with this iterative process.
This part of the worksheet enables you to efficiently perform a sequence of iterations to determine the velocity of a particle with known force. 
The rationale is as follows:

1. Guess a velocity, $U_{est}$.
2. Use the rationale in Section 3 to calculate the force, $F_{est}$, required to propel the particle at $U_{est}$.
3. Compare $F_{est}$ to the required force, and use the error to adjust the next guess of velocity up or down as needed.

Repeat until the necessary accuracy has been achieved.
With a little practice, you will be able to calculate velocity to within a percent or less error in a few iterations.

The input panel below has text boxes for the particle size, the actual force on the particle, and the fluid density and viscosity &ndash; and your guess at the velocity &ndash; as inputs. 
Its output is the force required to move the particle at the velocity you guessed.

In [6]:
mu5=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho5=widgets.FloatText(value=1028.,description = r"$\rho~~ (\frac{kg}{m^3})$")
U_guess=widgets.FloatText(value=0.001,description = r"$U_{guess}~~ (\frac{m}{s})$")
D5=widgets.FloatText(value=5.e-6,description = r"$D~~ (m)$")
F5=widgets.FloatText(value=1.e-11,description = r"$F~~ (N)$")

ui1c = widgets.VBox([mu5,F5,U_guess])
ui2c = widgets.VBox([rho5,D5])
ui3c = widgets.HBox([ui1c, ui2c])

def force5(rho5,U_guess,D5,mu5,F5):
    Re_guess=float(D5*abs(U_guess)*rho5/mu5)
    logRe=log(Re_guess,10.)
    logCd=CdRe(logRe)
    Cd_guess=10.**logCd
    F_guess=np.sign(U_guess)*0.5*rho5*pi/4.*D5**2*U_guess**2*Cd_guess
    #print('Particle density: rho_sphere = {:.4g} kg/m^3'.format(rho_sphere))
    print('Using guessed velocity, U_est = {:.4g}'.format(U_guess))
    print('Reynolds number: Re = {:.4g}'.format(Re_guess))
    print('\nForce required to move sphere at velocity U_est:')
    print('F_est = {:.4g}'.format(F_guess))

    print('\nF_est-F_actual = {:.4g}'.format(F_guess-F5))
    print('\nRelative error is {:.4g} %'.format(100.*(F_guess-F5)/F5))
  
out5 = widgets.interactive_output(force5,{'rho5':rho5,'U_guess':U_guess,'D5':D5,'mu5':mu5,'F5':F5})
display(ui3c,out5)

Output()

:::{figure} #rs1_1
:placeholder: Images/RS1_4.png
:::

### Velocity calculator #2
This calculator is an extension of Velocity Calculator #1. This calculator automatically does the iteration to find the $U$ which simultaneously satisfies the equations for $\mathcal{Re}$ and $C_d$. 
It also incorporates the calculation in the density calculator above, so that the input is not force directly but particle density.
The input panel below has text boxes for the particle size and density, and the fluid density and viscosity. Its output is the velocity of the corresponding particle.

In [7]:
def cd_error(r,F_D,rho,mu,U):
    """A function evaluating the error between the parameter U and the
    movement speed of a sphere with radius r subjected to force F_D in
    fluid with density rho and viscosity mu. The U for which this function
    evaluates to zero is the steady state movement speed. """
    if U==0.:
        return -F_D
    else:
        signU=np.sign(U)
        absU=abs(U)
        #print 'got here...',r,F_D,rho,mu,U
        logRe=log(2.*rho*r*absU/mu,10)
        logCd=CdRe(logRe)
        return (signU*0.5*rho*pi*r**2*U**2)*10**logCd - F_D

def U_sphere(r,F_D,rho,mu):
    """Steady-state movement velocity of a sphere in fluid, as determined by the
    root of the error metric cd_error. 
    """
    # Determine the range over which the numerical root-finding algorithm should operate:
    #Re_=2*rho*r*U./mu
    Urange=relist[-1]*mu/(2.*rho*r)
    Uerr = lambda u: cd_error(r,F_D,rho,mu,u)
    U=bisect(Uerr,-Urange,Urange, rtol=0.000001,full_output=False)
    return U

mu6=widgets.FloatText(value=1.376e-3,width=20,description = r"$\mu~~ (\frac{N s}{m^2})$")
rho_water=widgets.FloatText(value=1028.,description = r"$\rho~~ (\frac{kg}{m^3})$")
rho_sphere=widgets.FloatText(value=1105.,description = r"$\rho_{sphere}~~ (\frac{kg}{m^3})$")
#U_guess=widgets.FloatText(value=0.001,description = r"$U_{guess}~~ (\frac{m}{s})$")
D6=widgets.FloatText(value=5.e-6,description = r"$D~~ (m)$")
#F6=widgets.FloatText(value=1.e-11,description = r"$F~~ (N)$")

ui1d = widgets.VBox([mu6,rho_water])
ui2d = widgets.VBox([rho_sphere,D6])
ui3d = widgets.HBox([ui1d, ui2d])

def U6(rho_water,rho_sphere,D6,mu6):
    rr=D6/2.
    vol=float(4./3.*pi*rr**3)
    excess_mass=-(rho_sphere-rho_water)*vol # Keep the same sign convention as 
                                            # Het_labX.sws -- positive z is up
    g=9.81
    ff=g*excess_mass
    if rho_water==rho_sphere:
        print('External force is zero...')
        U_s=0.
    else:
        U_s=U_sphere(rr,ff,rho_water,mu6)
    print('Volume = {:.4g}'.format(vol))
    print('Excess mass = {:.4g}'.format(excess_mass))
    print('Gravity/buoyancy force = {:.4g}'.format(g*excess_mass))
    print('Reynolds number: Re = {:.4g}'.format(2.*rr*abs(U_s)*rho_water/mu6))
    print('U = {:.4g}'.format(U_s))
  
out6 = widgets.interactive_output(U6,{'rho_water':rho_water,'rho_sphere':rho_sphere,
                                      'D6':D6,'mu6':mu6})
display(ui3d,out6)

    #Flux_diff= 4 * pi * dd * rr * (C_w-C_s)
    #Sh = Sherwood(rr,dd,rho_water,mu,abs(U_s))
    #Flux_adv=float(Sh*Flux_diff)
    #print 'Sherwood number: Sh = ',Sh
    #print 'Diffusive flux: F_diff = ',float(Flux_diff)
    #print 'Advective flux: F_adv = ',float(Flux_adv)

Output()

:::{figure} #rs1_1
:placeholder: Images/RS1_5.png
:::